# Scrap them all !

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response


In [35]:
%qtconsole

In [2]:
holland_ayrault_1 = "https://fr.wikipedia.org/wiki/Gouvernement_Jean-Marc_Ayrault_(1)"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
page = request.Request(holland_ayrault_1, headers=header)
html = request.urlopen(page)
soup = BeautifulSoup(html, "lxml")

In [62]:
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="fr">
<head>
<meta charset="utf-8"/>
<title>Gouvernement Jean-Marc Ayrault (1) — Wikipédia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ = window.RLQ || []).push(function () {
mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Gouvernement_Jean-Marc_Ayrault_(1)","wgTitle":"Gouvernement Jean-Marc Ayrault (1)","wgCurRevisionId":123912507,"wgRevisionId":123912507,"wgArticleId":6295872,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article à illustrer Gouvernement","Catégorie Commons avec lien local différent sur Wikidata","Portail:Années 2010/Articles liés","Portail:XXIe siècle/Articles liés","Portail:Époque contemporaine/Articles liés","Portail:Politique française/Articles liés","Po

In [3]:
cells = soup.findAll("table", {"class": "wikitable sortable"})

In [4]:
for cell in cells:
    cell_tr = cell.findAll('tr')
    for c_tr in cell_tr:
        print(c_tr.findAll('a'))


[]
[<a class="image" href="/wiki/Fichier:Jean-Marc_Ayrault_-_mars_2012.jpg"><img alt="Jean-Marc Ayrault - mars 2012.jpg" data-file-height="924" data-file-width="618" height="120" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/be/Jean-Marc_Ayrault_-_mars_2012.jpg/80px-Jean-Marc_Ayrault_-_mars_2012.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/be/Jean-Marc_Ayrault_-_mars_2012.jpg/120px-Jean-Marc_Ayrault_-_mars_2012.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/be/Jean-Marc_Ayrault_-_mars_2012.jpg/160px-Jean-Marc_Ayrault_-_mars_2012.jpg 2x" width="80"/></a>, <a href="/wiki/Premier_ministre_fran%C3%A7ais" title="Premier ministre français">Premier ministre</a>, <a href="/wiki/Jean-Marc_Ayrault" title="Jean-Marc Ayrault">Jean-Marc Ayrault</a>, <a href="/wiki/Parti_socialiste_(France)" title="Parti socialiste (France)">PS</a>]
[]
[<a class="image" href="/wiki/Fichier:Fabius_4_f%C3%A9vrier_2013.jpg"><img alt="Fabius 4 février 2013.jpg" data-file-height="1719

In [5]:
cells[0].findAll('tr')[1].findAll('a')[2]['href']

'/wiki/Jean-Marc_Ayrault'

In [96]:
wiki_ministre = []
fonction_minitre = [] 
n_ministres = []
n_min = 0
for cell in cells:
    cells_href = cell.findAll("a", href=True)
    for c_href in cells_href:
        print(c_href.get_text())
        wiki_ministre.append(c_href['href']) # à filtrer
        fonction_minitre.append(c_href.get_text())
        n_ministres.append(n_min)
        n_min += 1


Premier ministre
Jean-Marc Ayrault
PS

Ministre des Affaires étrangères
Laurent Fabius
PS

Ministre de l'Éducation nationale
Vincent Peillon
PS

Garde des Sceaux
ministre de la Justice
Christiane Taubira
Walwari
PRG

Ministre de l'Économie, des Finances et du Commerce extérieur
Pierre Moscovici
PS

Ministre des Affaires sociales et de la Santé
Marisol Touraine
PS

Ministre de l'Égalité des territoires et du Logement
Cécile Duflot
EELV

Ministre de l'Intérieur
Manuel Valls
PS

Ministre de l'Écologie, du Développement durable et de l'Énergie
Nicole Bricq
PS

Ministre du Redressement productif
Arnaud Montebourg
PS

Ministre du Travail, de l'Emploi, de la Formation professionnelle et du Dialogue social
Michel Sapin
PS

Ministre de la Défense
Jean-Yves Le Drian
PS

Ministre de la Culture et de la Communication
Aurélie Filippetti
PS

Ministre de l'Enseignement supérieur et de la Recherche
Geneviève Fioraso
PS

Ministre des Droits des femmes
porte-parole du gouvernement
Najat Vallaud-Belkace

In [95]:
wiki_ministre[:9]  # mettre dans un DF et utiliser les ".jpg" pour séparer les ministres

[]

In [36]:
fonction_minitre[:9]

['',
 'Premier ministre',
 'Jean-Marc Ayrault',
 'PS',
 '',
 'Ministre des Affaires étrangères',
 'Laurent Fabius',
 'PS',
 '']

In [107]:
link_ministres_clean = [x for x in wiki_ministre if ('France' not in x) & 
                        ('jpg' not in x) & ('Fichier' not in x) & (len(x) <= 35) &
                        ('gauche' not in x) & ('Outre-' not in x) & ('Ville' not in (x)) &
                        ('Logement' not in x) & ('Budget' not in x)  & ('Walwari' not in x)]

# -- On vérifie qu'il y a bien 34 + 1 liens (nb ministres + PM)
assert len(link_ministres_clean) == 35